In [ ]:
import cv2
import os
import numpy as np

input_folder = r"C:\Users\abely\OneDrive\Desktop\tomography\tom_snimki"
output_folder = r"C:\Users\abely\OneDrive\Desktop\tomography\tom_snimki_id"


#https://pyimagesearch.com/2016/02/01/opencv-center-of-contour/


# Функция для нахождения ROI на изображении с помощью неоднородного серого круга
def find_roi(image):
    # Преобразуем изображение в оттенки серого
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Применяем адаптивную пороговую обработку для получения двоичного изображения
    _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # Находим контуры на двоичном изображении
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Если найдены контуры, выбираем наибольший
    if len(contours) > 0:
        roi = max(contours, key=cv2.contourArea)
        
        # Возвращаем координаты ограничивающего круга (x, y, radius) ROI
        (x, y), radius = cv2.minEnclosingCircle(roi)
        return (int(x), int(y), int(radius))
    
    return None

# Получаем список файлов с изображениями в папке
image_files = os.listdir(input_folder)

# Обрабатываем каждый файл
for file in image_files:
    
    image = cv2.imread(os.path.join(input_folder, file))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
    
        for c in contours:
            largest_contour = max(contours, key=cv2.contourArea)
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
            else:
                # Обработка случая, когда m00 равно нулю
                cX = 0
                cY = 0
            cv2.drawContours(image, [largest_contour], -1, (0, 255, 0), 2)
            cv2.circle(image, (cX, cY), 7, (255, 255, 255), -1)
            cv2.putText(image, "center", (cX - 20, cY - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            # Чтобы сохранить исходное изображение до обрезки
            cropped_image = image.copy()

            mask = np.zeros_like(image)
            cv2.drawContours(mask, [largest_contour], -1, (255, 255, 255), thickness=-1)

            # Применение битовой маски к изображению
            cropped_image = cv2.bitwise_and(cropped_image, mask)

        cv2.imshow("Cropped Image", cropped_image)
        cv2.imshow("Image", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()